In [ ]:
from google.colab import files
files.upload()

KeyboardInterrupt: 

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d ifigotin/imagenetmini-1000

In [ ]:
! unzip imagenetmini-1000.zip

Streaming output truncated to the last 5000 lines.
  inflating: imagenet-mini/train/n07875152/n07875152_2632.JPEG  
  inflating: imagenet-mini/train/n07875152/n07875152_2763.JPEG  
  inflating: imagenet-mini/train/n07875152/n07875152_3015.JPEG  
  inflating: imagenet-mini/train/n07875152/n07875152_3378.JPEG  
  inflating: imagenet-mini/train/n07875152/n07875152_3497.JPEG  
  inflating: imagenet-mini/train/n07875152/n07875152_3745.JPEG  
  inflating: imagenet-mini/train/n07875152/n07875152_4157.JPEG  
  inflating: imagenet-mini/train/n07875152/n07875152_461.JPEG  
  inflating: imagenet-mini/train/n07875152/n07875152_4704.JPEG  
  inflating: imagenet-mini/train/n07875152/n07875152_4789.JPEG  
  inflating: imagenet-mini/train/n07875152/n07875152_4848.JPEG  
  inflating: imagenet-mini/train/n07875152/n07875152_4940.JPEG  
  inflating: imagenet-mini/train/n07875152/n07875152_546.JPEG  
  inflating: imagenet-mini/train/n07875152/n07875152_5607.JPEG  
  inflating: imagenet-mini/train/n0787515

In [ ]:
import pandas as pd
import numpy as np
import os
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
# # creating a csv for the dataset
# base_folder_name = "imagenet-mini"
# df_list = []
# for folder in os.listdir(base_folder_name):
#     df = pd.DataFrame(columns=["idx",'image_path'])
#     for image_folder in os.listdir(os.path.join(base_folder_name,folder)):
#         for image_path in os.listdir(os.path.join(base_folder_name,folder,image_folder)):
#             df.loc[len(df)] = [len(df),os.path.join(base_folder_name,folder,image_folder,image_path)]
#     df_list.append(df)

In [ ]:
# train_df = df_list[0]
# test_df = df_list[1]

In [ ]:
train_df.to_csv('train.csv',index=False)
test_df.to_csv('test.csv', index=False)

In [ ]:
import pandas as pd
import numpy as np
import cv2

In [ ]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [ ]:
train_df.head()

,idx,image_path
0,0,imagenet-mini/train/n01496331/n01496331_32072....
1,1,imagenet-mini/train/n01496331/n01496331_3153.JPEG
2,2,imagenet-mini/train/n01496331/n01496331_11594....
3,3,imagenet-mini/train/n01496331/n01496331_5102.JPEG
4,4,imagenet-mini/train/n01496331/n01496331_694.JPEG


In [ ]:
train_df.iloc[0,1]

'imagenet-mini/train/n01496331/n01496331_32072.JPEG'

In [ ]:
train_df.head()

,idx,image_path
0,0,imagenet-mini/train/n01496331/n01496331_32072....
1,1,imagenet-mini/train/n01496331/n01496331_3153.JPEG
2,2,imagenet-mini/train/n01496331/n01496331_11594....
3,3,imagenet-mini/train/n01496331/n01496331_5102.JPEG
4,4,imagenet-mini/train/n01496331/n01496331_694.JPEG


In [ ]:
img = cv2.imread(train_df.iloc[0,1])
img.shape

(335, 500, 3)

In [ ]:
# cv2_imshow(img[:,:,0])

In [ ]:
def rgbTo313(lab_image):
  a_channel = lab_image[:,:,1].ravel()
  b_channel = lab_image[:,:,2].ravel()
  ab = np.column_stack((a_channel,b_channel))

  _, idx = nn_model.kneighbors(ab)
  idx = idx.ravel()
  h,w = lab_image.shape[:2]
  one_shot = np.zeros((h*w,313), dtype=np.float32)
  one_shot[np.arange(h*w),idx] = 1.0
  return one_shot.reshape((h,w,313))

def load_image(df):
  batch_size = len(df)
  bw_img_list = np.zeros((batch_size, 224, 224, 1), dtype=np.float32)
  color_img_list = np.zeros((batch_size, 224, 224, 313), dtype=np.float32)
  idx = 0
  for i, row in df.iterrows():
    img = cv2.imread(row[1])
    img = cv2.resize(img, (224,224))
    lab_image = rgb2lab(img)
    color_img_list[idx] = rgbTo313(lab_image)
    bw_img_list[idx] = (lab_image[:,:,0]/100.0).reshape(224,224,1)
    idx+=1
  return bw_img_list, color_img_list

In [ ]:
from skimage.color import rgb2lab
from sklearn.neighbors import NearestNeighbors

In [ ]:
ab_bins = np.load('pts_in_hull.npy')

In [ ]:
nn_model = NearestNeighbors(n_neighbors=1)
nn_model.fit(ab_bins)

NearestNeighbors(n_neighbors=1)

In [ ]:
train_df.shape

(34745, 2)

In [ ]:
processed_data = load_image(train_df)

<ipython-input-9-707da9100d51>:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  img = cv2.imread(row[1])
<ipython-input-9-707da9100d51>:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  img = cv2.imread(row[1])
<ipython-input-9-707da9100d51>:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  img = cv2.imread(row[1])
<ipython-input-9-707da9100d51>:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a fu

In [ ]:
def data_loader(df, batch_size):
  for i in range(0, len(df)//batch_size):
    batch_df = df.iloc[i*batch_size:(i+1)*batch_size,:]
    yield load_image(batch_df)

In [21]:
import pandas as pd
import numpy as np

In [ ]:
data_x = np.memmap("train_x.dat", dtype='float32', mode='r').reshape(-1, 224, 224, 1)

In [ ]:
from tensorflow.keras.layers import Dense, Input, Conv2D, BatchNormalization, UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.utils import Sequence
from tensorflow.keras.regularizers import l2
from tensorflow.keras.metrics import MeanSquaredError
from tensorflow.image import psnr
from tensorflow.keras.optimizers import Adam

In [ ]:
class CustomDataGenerator(Sequence):
  def __init__(self, df, batch_size, ab_bins, nn_model, shuffle=True):
    self.df = df
    self.batch_size = batch_size
    self.shuffle = shuffle
    self.ab_bins = ab_bins
    self.nn_model = nn_model

  def on_epoch_end(self):
    if(self.shuffle):
      self.df = self.df.sample(frac=1).reset_index(drop=True)

  def __len__(self):
    return self.df.shape[0]//self.batch_size

  def __getitem__(self,idx):
    batch_df = self.df.iloc[idx*self.batch_size:(idx+1)*self.batch_size,:]
    return x,y


In [ ]:
train_generator = CustomDataGenerator(train_df, 16, ab_bins, nn_model)
test_generator = CustomDataGenerator(test_df, 16, ab_bins, nn_model)

In [ ]:
input_layer = Input(shape=(224,224,1))
x = Conv2D(64, (3,3), activation='relu', padding='same', kernel_regularizer=l2)(input_layer)
x = BatchNormalization()(x)
x = Conv2D(128, (3,3), activation='relu', padding='same', kernel_regularizer=l2)(x)
x = BatchNormalization()(x)
x = Conv2D(256, (3,3), activation='relu', padding='same', kernel_regularizer=l2)(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3,3), activation='relu', padding='same', kernel_regularizer=l2)(x)
x = BatchNormalization()(x)
x = Conv2D(256, (3,3), activation='relu', padding='same', kernel_regularizer=l2)(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3,3), activation='relu', padding='same', kernel_regularizer=l2)(x)
x = BatchNormalization()(x)
x = Conv2D(64, (3,3), activation='relu', padding='same', kernel_regularizer=l2)(x)
x = BatchNormalization()(x)
output_layer = Conv2D(313, (1,1), activation='softmax', padding='same')(x)
model = Model(inputs=input_layer, outputs=output_layer)


In [ ]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 224, 224, 64)   │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 224, 224, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 224, 224, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 224, 224, 128)  │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 224, 224, 256)  │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 224, 224, 256)  │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 224, 224, 512)  │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 224, 224, 512)  │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 224, 224, 256)  │     1,179,904 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 224, 224, 256)  │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 224, 224, 128)  │       295,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 224, 224, 128)  │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 224, 224, 64)   │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 224, 224, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 224, 224, 313)  │        20,345 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,124,537 (11.92 MB)

 Trainable params: 3,121,721 (11.91 MB)

 Non-trainable params: 2,816 (11.00 KB)

In [ ]:
epochs = 20
lr = 0.01
optimizer = Adam(learning_rate=lr)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(train_generator, validation_data=test_generator, epochs=epochs)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20


KeyboardInterrupt: 